In [303]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist


In [304]:
NEURONS_COUNT = 10
EPOCH = 100

error = 0
learning_speed = 0.00001

In [312]:
(data_train, label_train), (data_test, label_test) = mnist.load_data()

In [313]:
data_train = data_train.reshape(data_train.shape[0], data_train.shape[1] * data_train.shape[2]) / 255
data_test = data_test.reshape(data_test.shape[0], data_test.shape[1] * data_test.shape[2]) / 255

In [314]:
class neuron:
    _SIZE_OF_WEIGHTS = 784

    def __init__(self, label: int):
        self._weights = (np.random.rand(self._SIZE_OF_WEIGHTS) - 0.5) * 0.06
        self._weights_correction = np.zeros(self._SIZE_OF_WEIGHTS)
        self._bias = 1
        self._label = label

    def get_output_value(self, inputed_value):
        return self._get_sigmoid(self._get_weighted_sum(inputed_value) + self._bias)
    
    def learn(self, inputed_value, inputed_label):
        self._change_weights_correction(inputed_value)

        if self._label != inputed_label:
            if self.get_output_value(inputed_value) >= self._label:
                self._weights += self._weights_correction
            
            return

        if self.get_output_value(inputed_value) < self._label:
            self._weights -= self._weights_correction
            return
            
    def _get_sigmoid(self, weighted_sum):
        return 1 / (1 + np.exp(-weighted_sum))
    
    def _get_sigmoid_diff(self, sigmoid):
        return (1 + sigmoid) * (1 - sigmoid)

    def _get_weighted_sum(self, inputed_value):
        return np.sum(np.multiply(self._weights, inputed_value))
    
    def _get_learning_speed_coefficient(self, sigmoid, sigmoid_diff):
        return -(self._label - sigmoid) * sigmoid_diff
    
    def _change_weights_correction(self,inputed_value):
        sigmoid = self.get_output_value(inputed_value)
        sigmoid_diff = self._get_sigmoid_diff(sigmoid)

        self._weights_correction -= learning_speed * self._get_learning_speed_coefficient(sigmoid, sigmoid_diff) * inputed_value


In [315]:
def get_neurons(count: int):
    return [neuron(i) for i in range(count)]

In [316]:
def get_error(output_value):
    return np.sum([0.5 * (desired_value - output_value) * (desired_value - output_value) for desired_value in range(NEURONS_COUNT)])

def change_error(output_value, neurons_count):
    error += get_error(output_value) / neurons_count

In [317]:
neurons = get_neurons(NEURONS_COUNT)

In [318]:
training_count = len(data_train)

In [319]:
for epoch in range(EPOCH):
    for neuron in neurons:
        for current_index in range(100):
            neuron.learn(data_train[current_index], label_train[current_index])

C:\Users\user\AppData\Local\Temp\ipykernel_6100\1483186857.py:28: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-weighted_sum))


In [320]:
neurons[4].get_output_value(data_test[4])

C:\Users\user\AppData\Local\Temp\ipykernel_6100\1483186857.py:28: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-weighted_sum))


0.0

In [321]:
label_test[4]

4